In [1]:
# Train the CNN Genre Classification Model#
# Last editted by Pu Zeng, 18/10/2023 #

# Install the library needed

In [2]:
!git clone https://github.com/Pu-Zeng/696-Milestone.git

Cloning into '696-Milestone'...
remote: Enumerating objects: 3047, done.
remote: Counting objects: 100% (3047/3047), done.
remote: Compressing objects: 100% (2554/2554), done.
remote: Total 3047 (delta 55), reused 3029 (delta 37), pack-reused 0
Receiving objects: 100% (3047/3047), 34.67 MiB | 22.89 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [3]:
!pip install pretty_midi
!pip install pypianoroll

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 5.0 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=fdb991997d6b9c012293f4c89a6f66475b793d57bd79d0211ec214cee2bb31df
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [4]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
# import mido
import string
import numpy as np
# from utilis import get_pianoroll_data
import pickle
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
# import mido
import string
import numpy as np
# from utilis import get_pianoroll_data
import pickle
from sklearn import preprocessing
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #Define device
input_dir = '../Toy_Dataset/adl-piano-midi/' #Please change
output_dir = './Processed_Data/'

# Preprocess Data

In [5]:
# Use our python script to preprocess the midi files into matrices
!mkdir Processed_Data
!python /content/696-Milestone/Data_Preprocessing/Piano_Genre_Classification.py /content/696-Milestone/Toy_Dataset/adl-piano-midi/ ./Processed_Data/
# It takes some time

All content numbers is 18
145it [01:26,  1.67it/s]
146it [01:17,  1.88it/s]
146it [01:25,  1.70it/s]
146it [01:01,  2.38it/s]
145it [01:30,  1.61it/s]
146it [01:14,  1.97it/s]
146it [01:18,  1.86it/s]
146it [01:11,  2.05it/s]
145it [01:17,  1.86it/s]


In [6]:
# Pick the midi files with more than 51.2s in the middle

from tqdm import tqdm
X = []
y = []
for i in tqdm(range(1,10)):
    file=open(output_dir+"music_data"+str(i)+".bin","rb")
    music_data = pickle.load(file) #保存list到文件
    file.close()
    for m in music_data:
        if m[2].shape[0]>=1012:
            X.append(m[2][500:1012,:])
            y.append(m[0])
X = np.array(X)

100%|██████████| 9/9 [00:03<00:00,  2.74it/s]


In [7]:
# The dataset is just a toy dataset, so there are far less labels than the whole Lakh dataset, we need to drop those will few samples
X_new = []
y_new = []
X = np.array(X)
y = np.array(y)
temp_y = np.array(y)
count_y_dict = {}
for i in np.unique(temp_y):
  if (temp_y==i).sum()>=3:
    X_new = X_new + list(X[temp_y==i])
    y_new = y_new + list(y[temp_y==i])
X = np.array(X_new)
y = np.array(y_new)
num_classes = len(np.unique(y))

In [8]:
# Transform labels into one-hot variables

le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

label_dict = {}
for cl in le.classes_:
    label_dict.update({cl:le.transform([cl])[0]})

In [9]:
num_classes

17

# Train the Model

In [10]:
# CNN Model

class CNNModel(nn.Module):
    def __init__(self, hiddenSize, outChannels, dropoutRate, activate):
        super().__init__()
        self.outChannels = outChannels
        self.activate = nn.Sigmoid() if activate == "Sigmoid" else nn.ReLU()
        self.conv1 = nn.Conv2d(1, 24, (10,1))
        self.pool = nn.MaxPool2d((2, 1))
        self.conv2 = nn.Conv2d(24, 48, (10,1))
        self.conv3 = nn.Conv2d(48, 96, (10,1))
        self.conv4 = nn.Conv2d(96, 192, (10,1))
        self.conv5 = nn.Conv2d(192, 192, (10,2))
        self.dense1 = nn.Linear(10668*16, hiddenSize)
        self.dropout = nn.Dropout(dropoutRate)
        self.dense2 = nn.Linear(hiddenSize, num_classes)

    def forward(self, x):
        x = self.pool(self.activate(self.conv1(x)))
        x = self.pool(self.activate(self.conv2(x)))
        # print(x.shape)
        x = self.dropout(self.pool(self.activate(self.conv3(x))))
        x = self.pool(self.activate(self.conv4(x)))
        x = self.pool(self.activate(self.conv5(x)))
        # print(x.shape)
        x = x.view(-1, 10668*16)
        x = self.dropout(self.activate(self.dense1(x)))
        return self.dense2(x)

# Number of neurons in the first fully-connected layer
hiddenSize = 300
# Number of feature filters in second convolutional layer
numFilters = 25
# Dropout rate
dropoutRate = 0.3
# Activation function
activation = "ReLU"
# Learning rate
learningRate = 0.001
# Momentum for SGD optimizer
momentum = 0.9
# Number of training epochs
numEpochs = 50

In [11]:
# Train and Validation function

from tqdm import tqdm
def train_epoch(cnn,device,dataloader,loss_fn,optimizer):
    cnn.train()
    cnnRunningLoss = 0
    total = 0
    R2 = 0
    cnnCorrect=0
    total1=0
    for i, (inputs, labels) in enumerate(dataloader, 0):
        optimizer.zero_grad()
        inputs = inputs.to(device).reshape(-1,1,512,128) # Transform the input shape

        labels = labels.to(device)
        # Forward propagation
        cnnOutputs = cnn(inputs)
        # print(cnnOutputs.shape)
        l2_lambda = 0.05
        l2_reg = torch.tensor(0.).to(device)
        for param in cnn.parameters():
            l2_reg += torch.norm(param)
        # Backpropagation
        cnnLoss = criterion(cnnOutputs, labels)+l2_reg*l2_lambda
        cnnLoss.backward()
        # Gradient update
        optimizer.step()
        total += 1
        total1+=labels.size(0)
        cnnRunningLoss += cnnLoss.item()
        _, cnnPredicted = torch.max(cnnOutputs.data, 1)
        cnnCorrect += (cnnPredicted == labels).sum().item()
    return cnnRunningLoss/total, cnnCorrect/total1

def valid_epoch(cnn,device,dataloader,loss_fn):
    cnn.eval()
    totalLoss = 0
    total = 0
    total1 = 0
    R2 = 0
    cnnLoss = 0
    cnnCorrect=0
    for inputs, labels in dataloader:
        inputs = inputs.to(device).reshape(-1,1,512,128)
        labels = labels.to(device)
        cnnOutputs = cnn(inputs)
        cnnLoss = criterion(cnnOutputs, labels)
        _, cnnPredicted = torch.max(cnnOutputs.data, 1)
        total += labels.size(0)
        total1 +=1
        totalLoss += cnnLoss.item()
        cnnCorrect += (cnnPredicted == labels).sum().item()
    accuracy = cnnCorrect / total
    cnn.train()
    return totalLoss/total1, accuracy

In [ ]:
# Train the model

import sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
X_train_v, X_test, y_train_v, y_test = train_test_split(X, y, test_size=0.2, random_state=99, stratify=y) #First split, so there is no data leakage problems

# Transform data into DataSet
X_train_v = torch.as_tensor(X_train_v, dtype=torch.float) # an alternative to torch.from_numpy
y_train_v = torch.as_tensor(y_train_v, dtype=torch.float)
X_test = torch.as_tensor(X_test, dtype=torch.float)
y_test = torch.as_tensor(y_test, dtype=torch.float)
splits=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
train_dataset = TensorDataset(X_train_v, y_train_v)
test_dataset = TensorDataset(X_test, y_test)
batch_size=128
models = []
history = {'fold':[], 'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

#K-Fold validation
for fold, (train_idx,val_idx) in enumerate(splits.split(X_train_v,y_train_v)):
    print('Fold {}'.format(fold + 1))
    t = pd.DataFrame(y_train_v[train_idx], columns = ['class']).reset_index()
    idx = []

    #Resampling to solve the unbalanced problem
    y_train = []
    for i in range(0,num_classes):
        # if i!=label_dict['Unknown']:
      idx += list(t[t['class']==i].sample(50,replace=True)['index'])
      y_train += [i]*50
    X_train = X_train_v[train_idx][idx]

    t = pd.DataFrame(y_train_v[val_idx], columns = ['class']).reset_index()
    idx = []
    y_val = []
    for i in range(0,num_classes):
        # if i!=label_dict['Unknown']:
      idx += list(t[t['class']==i].sample(5,replace=True)['index'])
      y_val += [i]*5
    X_val =  X_train_v[val_idx][idx]

    X_train = torch.as_tensor(X_train, dtype=torch.float) # an alternative to torch.from_numpy
    y_train = torch.as_tensor(y_train, dtype=torch.float).type(torch.LongTensor)
    X_val = torch.as_tensor(X_val, dtype=torch.float)
    y_val = torch.as_tensor(y_val, dtype=torch.float).type(torch.LongTensor)

    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_val, y_val)

    from torch.utils.data import Dataset, DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    model = CNNModel(hiddenSize, numFilters, dropoutRate, activation).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(list(model.parameters()), lr=learningRate, momentum=momentum)
    # optimizer = torch.optim.Adam(list(model.parameters()), lr=learningRate)
    best_test = -np.inf
    best_model = None # Save the best model
    #Train the model
    for epoch in range(numEpochs):
        train_loss, train_R2=train_epoch(model,device,train_dataloader,criterion,optimizer)
        test_loss, test_R2=valid_epoch(model,device,test_dataloader,criterion)
        history['fold'].append(fold)
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_R2)
        history['test_acc'].append(test_R2)
        if test_R2>best_test:
            test_test = test_R2
            best_model = model
        print('Train Loss: {}, Train_acc: {}, Test Loss: {}, Test acc: {}'.format(train_loss, train_R2, test_loss, test_R2))
    models.append([best_model])

Fold 1
Train Loss: 4.394462797376844, Train_acc: 0.3305882352941176, Test Loss: 2.6474756002426147, Test acc: 0.18823529411764706
Train Loss: 3.3290953945230553, Train_acc: 0.6717647058823529, Test Loss: 2.939752995967865, Test acc: 0.32941176470588235
Train Loss: 2.7108905756915056, Train_acc: 0.8458823529411764, Test Loss: 3.8783851067225137, Test acc: 0.36470588235294116
Train Loss: 2.4717595356482045, Train_acc: 0.9117647058823529, Test Loss: 4.113501826922099, Test acc: 0.36470588235294116
Train Loss: 2.4127669820079096, Train_acc: 0.9258823529411765, Test Loss: 4.882371187210083, Test acc: 0.36470588235294116
Train Loss: 2.3541754395873458, Train_acc: 0.9435294117647058, Test Loss: 4.961639483769734, Test acc: 0.4
Train Loss: 2.271095306785018, Train_acc: 0.9505882352941176, Test Loss: 5.201759735743205, Test acc: 0.35294117647058826
Train Loss: 2.323452772917571, Train_acc: 0.9411764705882353, Test Loss: 5.041255871454875, Test acc: 0.36470588235294116
Train Loss: 2.247440682517

# Evaluaton

In [ ]:
#Calculate the average accuray of the 5 models

from torch.utils.data import Dataset, DataLoader
X_train_v, X_test, y_train_v, y_test = train_test_split(X, y, test_size=0.2, random_state=99, stratify=y)
t = pd.DataFrame(y_test, columns = ['class']).reset_index()
criterion = nn.CrossEntropyLoss()
idx = []
y_test = []
with torch.no_grad():
  for i in range(0,17):
      # if i!=label_dict['Unknown']:
    idx += list(t[t['class']==i].sample(20,replace=True)['index'])
    y_test += [i]*20
  X_test =  X_test[idx]
  X_test = torch.as_tensor(X_test, dtype=torch.float)
  y_test = torch.as_tensor(y_test, dtype=torch.float).type(torch.LongTensor)

  test_dataset = TensorDataset(X_test, y_test)
  test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)
  acc = []
  for best_model in models:
    acc.append(valid_epoch(best_model[0],device,test_dataloader,criterion)[1])

In [ ]:
np.mean(acc)

In [ ]:
# Calculate the accuracy with majority vote
outputs = []
with torch.no_grad():
  for model in models:
    model[0].eval()
    model_output = []
    label_= []
    model[0].to(device)
    for inputs, labels in test_dataloader:
        inputs = inputs.reshape(-1,1,512,128).to(device)
        labels = labels.to(device)
        cnnOutputs = model[0](inputs)
        model_output.append(cnnOutputs)
        label_.append(labels)
        # del inputs
    outputs.append(torch.vstack(model_output))

In [ ]:
shape = outputs[0].shape
labels = torch.hstack(label_)
cnnresult = torch.vstack(outputs).reshape(-1,shape[0],shape[1]).mean(axis=0).to('cpu')
_, cnnPredicted = torch.max(cnnresult.data, 1)
cnnCorrect = (cnnPredicted.detach().numpy() == labels.to('cpu').detach().numpy()).sum().item()
print(cnnCorrect/len(labels)*100)
from sklearn.metrics import confusion_matrix
confu_m = confusion_matrix(labels.to('cpu'), cnnPredicted.to('cpu'))
import seaborn as sns
sns.heatmap(confu_m,square=False)

In [ ]:
# Show the Precision, Recall, F1 scores of each classes
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
scores = pd.DataFrame(precision_recall_fscore_support(cnnPredicted.to('cpu'), labels.to('cpu'), labels = list(range(17))),index=['precision','recall','f1','support'], columns = list(label_dict)[:17])
scores

In [ ]:
# Print the label_dict
label_dict = {}
for cl in le.classes_:
    label_dict.update({cl:le.transform([cl])[0]})
label_dict